# Imports

In [1]:
import requests as rq
import pandas as pd
import os
import numpy as np
import json
from dotenv import load_dotenv
from sqlalchemy import create_engine
import copy

In [2]:
def show_cols(dict_):
    for cat in dict_.keys():
        print(f'\n\nCategory: {cat}')
        print(*[ i for i in dict_[cat].columns], sep = '\n')

# Definitions

In [3]:
base_urls = {
    "films": "https://swapi.dev/api/films/",
    "people": "https://swapi.dev/api/people/",
    "planets": "https://swapi.dev/api/planets/",
    "species": "https://swapi.dev/api/species/",
    "starships": "https://swapi.dev/api/starships/",
    "vehicles": "https://swapi.dev/api/vehicles/"
}

In [4]:
categories = list(base_urls.keys())
categories

['films', 'people', 'planets', 'species', 'starships', 'vehicles']

Each category has different fields that contain information in the form of an url. I will extract the page id from those fields for each category. 

In [5]:
fields = {
    'films' : ["characters", "planets", "starships", "vehicles", "species"],
    'people' : ["homeworld", "films", "species", "vehicles", "starships"],
    'planets' : ['residents', 'films'],
    'species' : ['people', 'films', 'homeworld'],
    'vehicles' : ['pilots', 'films'],
    'starships' : ['pilots', 'films']
    }

# Consume the API

In [6]:
def scrape_category(url):

    # skip the next url for the first page of pager
    next = url    
    items_list = []

    while next:
        # get the content of the url
        response = rq.get(next)

        # success
        if response.status_code == 200:
            content = response.json()
        elif response.status_code == 404:
            print(f'{url} not found!')
            return
        
        next = content['next']
        items = content['results']

        for item in items:
                    
            # remove created and edited fields
            try:
                del(item['created'])
                del(item['edited'])
            except:
                pass
            
            items_list.append(item)            

    return items_list

## Scrape all the categories and store in *starwars_raw.json*

In [7]:
if not os.path.exists('./data/starwars_raw.json'):
    raw_dict = {cat : scrape_category(base_urls[cat]) for cat in categories}
    
    # store into a json file
    with open('./data/starwars_raw.json', 'w') as file:
        json.dump(raw_dict, file, indent=4 )
    print('Content from Star Wars API stored in a json file!')

else:
    print('The content already exists in a json file!')
    with open('starwars_scrapped.json', 'r') as file:
        raw_dict = json.load(file)

The content already exists in a json file!


Function to process the information of an item from a category.
Ex. one character, one planet or one film.

In [8]:
def process_item(item, fields):

    # create a copy of the item dictionary
    item = copy.deepcopy(item)

    # parse the links from starships, vehicles and species
    for field in fields:
        id_values = []
                        
        if item[field]:  # if the field is not empty
            
            # parse the homeworld (just a single string value)
            if field == 'homeworld':
                # get the homeworld id
                item['homeworld'] = int(item['homeworld'].split('/')[-2])

            else:
                for link in item[field]:

                    # parse the id value in the link    
                    id_values.append(int(link.split('/')[-2]))

                # add the id values into the corresponding field key
                # convert list into tuple, as tuples are hashable
                item[field] = tuple(id_values)

                # each character belongs to only 1 species
                # that way, the species contains
        
        # if species is an empty list, it's a human character
        elif field == 'species' and not item['species']:
            item[field] = 1
                    
        # field has no values (empty list)
        else:
            item[field] = ()
                
    # remove created and edited fields
    try:
        del(item['created'])
        del(item['edited'])
    except:
        pass

    return item

# Store the processed data

In [9]:
if not os.path.exists('./data/starwars_processed_items.json'):
    # dictionary to store the processed categories
    processed_dict = {}
    
    # process each item for all the categories
    for k,v in raw_dict.items():
        items_processed = []
        for item in v:
            try:
                items_processed.append(process_item(item, fields[k]))
                processed_dict[k] = items_processed
            except:
                print(f'Error in {k}')
    
    # store the information in a json file
    with open('./data/starwars_processed_items.json', 'w') as file:
        json.dump(processed_dict, file, indent = 4)

# the file already exists, so load it
else:
    with open('./data/starwars_processed_items.json', 'r') as file:
        processed_dict= json.load(file)

        # convert lists into tuples after reading from json file
        for cat in categories:
            for item in processed_dict[cat]:
                for field in fields[cat]:
                    try:
                        item[field] = tuple(item[field])
                    # the content of the field is an integer
                    # and not a list. Cannot create a tuple from
                    # an integer using tuple(int)
                    except:
                        pass  # do not convert into a tuple, leave it as integer

    print('Processed data already existed, so the *categories_dict_processed* dictionary will be created from json file.')

Processed data already existed, so the *categories_dict_processed* dictionary will be created from json file.


# Dataframes

## Create the dataframes
Create a dictionary to store the dataframes from each category

In [10]:
dataframes = dict.fromkeys(categories)

In [11]:
for cat in categories:
    df = pd.DataFrame(processed_dict[cat])
    df['id'] = df.index + 1

    # rename columns to add '_id' to the "fields"
    rename_dict = {field : f'{field}_id' for field in fields[cat]}
    rename_dict.update({'id' : f'{cat}_id'})
    df.rename(columns = rename_dict, inplace = True)

    # reorder the columns to place id in first place
    all_columns_but_cat_id = [col for col in df.columns if col != f'{cat}_id']
    sorted_columns = [f'{cat}_id'] + all_columns_but_cat_id
    dataframes[cat] = df[sorted_columns]
#df.to_csv('./data/starwars_characters.csv', index = False)

## Rename some columns
Make some renamings to the column names

In [12]:
col_rename_dict= {
    
    'films': {
        'characters_id': 'character_id',
        'films_id': 'film_id',
        'planets_id': 'planet_id',
        'episode_id': 'episode',
        'starships_id': 'starship_id',
        'vehicles_id': 'vehicle_id',
    },
    
    'people': {
        'people_id': 'character_id',
        'films_id': 'film_id',
        'vehicles_id': 'vehicle_id',
        'starships_id': 'starship_id'
    },

    'planets': {
        'planets_id': 'planet_id',
        'films_id': 'film_id'
    },

    'species': {
        'people_id': 'character_id',
        'films_id': 'film_id'
    },

    'vehicles': {
        'pilots_id': 'pilot_id',
        'vehicles_id': 'vehicle_id',
        'films_id': 'film_id'
    },

    'starships': {
        'pilots_id': 'pilot_id',
        'starships_id': 'starship_id',
        'films_id': 'film_id'
    }
}

In [13]:
for cat in categories:
    dataframes[cat].rename(columns=col_rename_dict[cat], inplace=True)

## Clean the datasets

### Clean people

In [14]:
df = dataframes['people']
df.sample(5)

,character_id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld_id,film_id,species_id,vehicle_id,starship_id,url
75,76,San Hill,191,unknown,none,grey,gold,unknown,male,57,"(5,)","(34,)",(),(),https://swapi.dev/api/people/77/
72,73,Jocasta Nu,167,unknown,white,fair,blue,unknown,female,9,"(5,)","(1,)",(),(),https://swapi.dev/api/people/74/
74,75,Wat Tambor,193,48,none,"green, grey",unknown,unknown,male,56,"(5,)","(33,)",(),(),https://swapi.dev/api/people/76/
59,60,Cordé,157,unknown,brown,light,brown,unknown,female,8,"(5,)",1,(),(),https://swapi.dev/api/people/61/
28,29,Wicket Systri Warrick,88,20,brown,brown,brown,8BBY,male,7,"(3,)","(9,)",(),(),https://swapi.dev/api/people/30/


In [15]:
df.dtypes

character_id     int64
name            object
height          object
mass            object
hair_color      object
skin_color      object
eye_color       object
birth_year      object
gender          object
homeworld_id     int64
film_id         object
species_id      object
vehicle_id      object
starship_id     object
url             object
dtype: object

In [16]:
df.mass = df.mass.replace('unknown', np.nan)
df.mass = df.mass.str.replace(',', '', regex=False)
df.mass = df.mass.astype('float') 

In [17]:
df.height = df.height.replace('unknown', np.nan).astype('float')

In [18]:
df.hair_color = df.hair_color.replace('n/a', np.nan)

In [19]:
df.birth_year = df.birth_year.str.replace('BBY', ' BBY')

### Clean films

In [20]:
df = dataframes['films']
df.sample(5)

,film_id,title,episode,opening_crawl,director,producer,release_date,character_id,planet_id,starship_id,vehicle_id,species_id,url
3,4,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,"(2, 3, 10, 11, 16, 20, 21, 32, 33, 34, 35, 36,...","(1, 8, 9)","(31, 32, 39, 40, 41)","(33, 34, 35, 36, 37, 38, 42)","(1, 2, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19, ...",https://swapi.dev/api/films/4/
5,6,Revenge of the Sith,3,War! The Republic is crumbling\r\nunder attack...,George Lucas,Rick McCallum,2005-05-19,"(1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 20, 21, ...","(1, 2, 5, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19)","(2, 32, 48, 59, 61, 63, 64, 65, 66, 68, 74, 75)","(33, 50, 53, 56, 60, 62, 67, 69, 70, 71, 72, 7...","(1, 2, 3, 6, 15, 19, 20, 23, 24, 25, 26, 27, 2...",https://swapi.dev/api/films/6/
4,5,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,"(2, 3, 6, 7, 10, 11, 20, 21, 22, 33, 35, 36, 4...","(1, 8, 9, 10, 11)","(21, 32, 39, 43, 47, 48, 49, 52, 58)","(4, 44, 45, 46, 50, 51, 53, 54, 55, 56, 57)","(1, 2, 6, 12, 13, 15, 28, 29, 30, 31, 32, 33, ...",https://swapi.dev/api/films/5/
0,1,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","(1, 2, 3)","(2, 3, 5, 9, 10, 11, 12, 13)","(4, 6, 7, 8)","(1, 2, 3, 4, 5)",https://swapi.dev/api/films/1/
2,3,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,"(1, 2, 3, 4, 5, 10, 13, 14, 16, 18, 20, 21, 22...","(1, 5, 7, 8, 9)","(2, 3, 10, 11, 12, 15, 17, 22, 23, 27, 28, 29)","(8, 16, 18, 19, 24, 25, 26, 30)","(1, 2, 3, 5, 6, 8, 9, 10, 15)",https://swapi.dev/api/films/3/


In [21]:
df.species_id.unique()

array([(1, 2, 3, 4, 5), (1, 2, 3, 6, 7), (1, 2, 3, 5, 6, 8, 9, 10, 15),
       (1, 2, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27),
       (1, 2, 6, 12, 13, 15, 28, 29, 30, 31, 32, 33, 34, 35),
       (1, 2, 3, 6, 15, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 33, 34, 35, 36, 37)],
      dtype=object)

In [22]:
df.dtypes

film_id           int64
title            object
episode           int64
opening_crawl    object
director         object
producer         object
release_date     object
character_id     object
planet_id        object
starship_id      object
vehicle_id       object
species_id       object
url              object
dtype: object

In [23]:
df.release_date = pd.to_datetime(df.release_date)

### Clean planets

In [24]:
df = dataframes['planets']
df.sample(5)

,planet_id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents_id,film_id,url
56,57,Muunilinst,28,412,13800,temperate,1,"plains, forests, hills, mountains",25,5000000000,"(77,)",(),https://swapi.dev/api/planets/57/
27,28,unknown,0,0,0,unknown,unknown,unknown,unknown,unknown,"(20, 23, 29, 32, 75)",(),https://swapi.dev/api/planets/28/
3,4,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100,unknown,(),"(2,)",https://swapi.dev/api/planets/4/
45,46,Tholoth,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,(),(),https://swapi.dev/api/planets/46/
20,21,Eriadu,24,360,13490,polluted,1 standard,cityscape,unknown,22000000000,"(12,)",(),https://swapi.dev/api/planets/21/


In [25]:
df.dtypes

planet_id           int64
name               object
rotation_period    object
orbital_period     object
diameter           object
climate            object
gravity            object
terrain            object
surface_water      object
population         object
residents_id       object
film_id            object
url                object
dtype: object

In [26]:
for col in ['rotation_period', 'orbital_period', 'diameter', 'surface_water', 'population']:
    df[col] = df[col].replace('unknown', np.nan).astype('float')

In [27]:
df.population = df.population / 1E6
df.rename(columns = {'population' : 'population_millions'}, inplace = True)

In [28]:
df.gravity.unique()

array(['1 standard', '1.1 standard', 'N/A',
       '1.5 (surface), 1 standard (Cloud City)', '0.85 standard',
       '0.9 standard', '0.56 standard', '0.75 standard', 'unknown',
       '0.62 standard', '1', '1.56', '0.9', '0.98'], dtype=object)

In [29]:
df.gravity = df.gravity.str.replace(' standard', '').str.replace(df.gravity[5], '1.5')
df.gravity = df.gravity.replace('unknown', np.nan)

### Clean species

In [30]:
df = dataframes['species']
df.sample(5)

,species_id,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld_id,language,character_id,film_id,url
11,12,Gungan,amphibian,sentient,190,"brown, green",none,orange,unknown,8,Gungan basic,"(36, 37, 38)","(4, 5)",https://swapi.dev/api/species/12/
31,32,Kaminoan,amphibian,sentient,220,"grey, blue",none,black,80,10,Kaminoan,"(72, 73)","(5,)",https://swapi.dev/api/species/32/
5,6,Yoda's species,mammal,sentient,66,"green, yellow","brown, white","brown, green, yellow",900,28,Galactic basic,"(20,)","(2, 3, 4, 5, 6)",https://swapi.dev/api/species/6/
34,35,Togruta,mammal,sentient,180,"red, white, orange, yellow, green, blue",none,"red, orange, yellow, green, blue, black",94,58,Togruti,"(78,)","(5, 6)",https://swapi.dev/api/species/35/
35,36,Kaleesh,reptile,sentient,170,"brown, orange, tan",none,yellow,80,59,Kaleesh,"(79,)","(6,)",https://swapi.dev/api/species/36/


In [31]:
df.dtypes

species_id           int64
name                object
classification      object
designation         object
average_height      object
skin_colors         object
hair_colors         object
eye_colors          object
average_lifespan    object
homeworld_id        object
language            object
character_id        object
film_id             object
url                 object
dtype: object

In [32]:
df.average_height = df.average_height.replace('unknown', np.nan).replace('n/a', np.nan).astype('float')

In [33]:
df.average_lifespan = df.average_lifespan.replace('unknown', np.nan).replace('indefinite', 9999).astype('float')

In [34]:
df.loc[1, 'homeworld_id'] = np.nan
df.homeworld_id = df.homeworld_id.astype('float')

### Clean vehicles

In [35]:
df = dataframes['vehicles']
df.sample(5)

,vehicle_id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,pilot_id,film_id,url
4,5,Snowspeeder,t-47 airspeeder,Incom corporation,unknown,4.5,650,2,0,10,none,airspeeder,"(1, 18)","(2,)",https://swapi.dev/api/vehicles/14/
27,28,Flitknot speeder,Flitknot speeder,Huppla Pasa Tisc Shipwrights Collective,8000,2,634,1,0,unknown,unknown,speeder,"(67,)","(5,)",https://swapi.dev/api/vehicles/55/
8,9,Storm IV Twin-Pod cloud car,Storm IV Twin-Pod,Bespin Motors,75000,7,1500,2,0,10,1 day,repulsorcraft,(),"(2,)",https://swapi.dev/api/vehicles/20/
14,15,Multi-Troop Transport,Multi-Troop Transport,Baktoid Armor Workshop,138000,31,35,4,112,12000,unknown,repulsorcraft,(),"(4,)",https://swapi.dev/api/vehicles/34/
17,18,C-9979 landing craft,C-9979 landing craft,Haor Chall Engineering,200000,210,587,140,284,1800000,1 day,landing craft,(),"(4,)",https://swapi.dev/api/vehicles/37/


In [36]:
df.cost_in_credits = df.cost_in_credits.replace('unknown', np.nan)
df.cost_in_credits = df.cost_in_credits.astype('float')

In [37]:
for col in ['max_atmosphering_speed' ,'crew', 'passengers', 'cargo_capacity']:
    df[col] = df[col].replace('unknown', np.nan).replace('none', np.nan)
    df[col] = df[col].astype('float')

In [38]:
df.length = df.length.replace('unknown', np.nan)
df.length = df.length.astype('float')

In [39]:
df.consumables = df.consumables.replace('0', 'none')

### Clean starships

In [40]:
df = dataframes['starships']
df.sample(5)

,starship_id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilot_id,film_id,url
23,24,Jedi starfighter,Delta-7 Aethersprite-class interceptor,Kuat Systems Engineering,180000,8,1150,1,0,60,7 days,1.0,unknown,Starfighter,"(10, 58)","(5, 6)",https://swapi.dev/api/starships/48/
35,36,V-wing,Alpha-3 Nimbus-class V-wing starfighter,Kuat Systems Engineering,102500,7.9,1050,1,0,60,15 hours,1.0,unknown,starfighter,(),"(6,)",https://swapi.dev/api/starships/75/
13,14,Calamari Cruiser,MC80 Liberty type Star Cruiser,Mon Calamari shipyards,104000000,1200,n/a,5400,1200,unknown,2 years,1.0,60,Star Cruiser,(),"(3,)",https://swapi.dev/api/starships/27/
0,1,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,30-165,600,3000000,1 year,2.0,60,corvette,(),"(1, 3, 6)",https://swapi.dev/api/starships/2/
29,30,Republic attack cruiser,Senator-class Star Destroyer,"Kuat Drive Yards, Allanteen Six shipyards",59000000,1137,975,7400,2000,20000000,2 years,1.0,unknown,star destroyer,(),"(6,)",https://swapi.dev/api/starships/63/


In [41]:
df.loc[0, 'crew'] = 165

In [42]:
for col in ['cost_in_credits', 'length', 'max_atmosphering_speed', 'crew', 'passengers', 'cargo_capacity', 'hyperdrive_rating', 'MGLT']:
    try:
        df[col] = df[col].replace('unknown', np.nan).replace('none', np.nan).replace('n/a', np.nan)
        df[col] = df[col].str.replace(',', '', regex = False).str.replace('km', '')
        #df[col] = df[col].astype('float')
    except Exception as e:
        print(f'Error in {col}: {e}')

In [43]:
for col in ['cost_in_credits', 'length', 'max_atmosphering_speed', 'crew', 'passengers', 'cargo_capacity', 'hyperdrive_rating', 'MGLT']:
    try:
        df[col] = df[col].astype('float')
    except:
        print(f'error with {col}')

## Export clean datasets into csv files

In [44]:
data_path = './data'
for cat in categories:
    filename = f'{cat}_dataframe.csv'
    if os.path.exists(f'{data_path}/csv/{filename}'):
        print(f'File {filename} already exist!')
        pass
    else:
        os.makedirs(f'{data_path}/csv/', exist_ok=True)
        df = dataframes[cat]
        df.to_csv(f'{data_path}/csv/{cat}_dataframe.csv', index = False)
print(f'Dataframes of each normalized category are stored in {data_path}/csv/ as csv files!')

File films_dataframe.csv already exist!
File people_dataframe.csv already exist!
File planets_dataframe.csv already exist!
File species_dataframe.csv already exist!
File starships_dataframe.csv already exist!
File vehicles_dataframe.csv already exist!
Dataframes of each normalized category are stored in ./data/csv/ as csv files!


# Junction tables

(many-to-many relationships in the database)

1. **films_people_junction**: Links films to the characters that appeared in them.

    - character_id: Foreign Key referencing the `people` table.
    - film_id: Foreign Key referencing the films table.

2. **films_planets_junction**: Links films to the planets that appeared in them.

    - planet_id: Foreign Key referencing the `planets` table.
    - film_id: Foreign Key referencing the `films` table.

3. **films_starships_junction**: Links films to the starships that appeared in them.

    - starship_id: Foreign Key referencing the `starships` table.
    - film_id: Foreign Key referencing the `films` table.

4. **films_vehicles_junction**: Links films to the vehicles that appeared in them.

    - vehicle_id: Foreign Key referencing the `vehicles` table.
    - film_id: Foreign Key referencing the `films` table.

5. **films_species_junction**: Links films to the species that appeared in them.

    - species_id: Foreign Key referencing the `species` table.
    - film_id: Foreign Key referencing the `films` table.

6. **people_starships_junction**: Links people (pilots) to the starships they have piloted.

    - character_id: Foreign Key referencing the `people` table.
    - starship_id: Foreign Key referencing the `starships` table.

7. **people_vehicles_junction**: Links people (pilots) to the vehicles they have piloted.

    - character_id: Foreign Key referencing the `people` table.
    - vehicle_id: Foreign Key referencing the vehicles table.

In [45]:
junction_tables = [
    'people_vehicles_junction', 
    'people_starships_junction',
    
    'films_people_junction',
    'films_planets_junction',
    'films_starships_junction',
    'films_vehicles_junction',
    'films_species_junction'
    ]

junction_tables_dict = {i:None for i in junction_tables}

## Junction tables for films:

In [83]:
data = dataframes['films'].loc[:, ['character_id', 'species_id', 'planet_id', 'film_id', 'vehicle_id', 'starship_id']]

# junction table for people and films
junction_tables_dict['films_people_junction'] = (data.explode('character_id')
                                                 .explode('film_id')
                                                 .drop(['species_id', 'planet_id', 'vehicle_id', 'starship_id'], axis = 1)
                                                 .dropna() )

# junction table for planets and films
junction_tables_dict['films_planets_junction'] = (data.explode('planet_id')
                                                  .explode('film_id')
                                                  .drop(['species_id', 'character_id', 'vehicle_id', 'starship_id'], axis = 1)
                                                  .dropna() )

# junction table for people and films
junction_tables_dict['films_species_junction'] = (data.explode('species_id')
                                                  .drop(['character_id', 'planet_id', 'vehicle_id', 'starship_id'], axis = 1)
                                                  .dropna() )

# junction table for people and vehicles
junction_tables_dict['films_vehicles_junction'] = (data.explode('vehicle_id')
                                                   .drop(['character_id', 'planet_id', 'species_id', 'starship_id'], axis = 1)
                                                   .drop_duplicates()
                                                   .dropna() )

# junction table for people and starships
junction_tables_dict['films_starships_junction'] = (data.explode('starship_id')
                                                    .drop(['character_id', 'planet_id', 'species_id', 'vehicle_id'], axis = 1)
                                                    .drop_duplicates()
                                                    .dropna() )

## Junction tables for people:

In [85]:
data = dataframes['people'].loc[:, ['character_id', 'film_id', 'vehicle_id', 'starship_id']]

# junction table for people and vehicles
junction_tables_dict['people_vehicles_junction'] = (data.explode('vehicle_id')
                                                    .explode('character_id')
                                                    .drop(['film_id', 'starship_id'], axis = 1)
                                                    .drop_duplicates()
                                                    .dropna() )

# junction table for people and starships
junction_tables_dict['people_starships_junction'] = (data.explode('starship_id')
                                                     .explode('character_id')
                                                     .drop(['film_id', 'vehicle_id'], axis = 1)
                                                     .drop_duplicates()
                                                     .dropna() )

In [49]:
dataframes['films']

,film_id,title,episode,opening_crawl,director,producer,release_date,character_id,planet_id,starship_id,vehicle_id,species_id,url
0,1,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","(1, 2, 3)","(2, 3, 5, 9, 10, 11, 12, 13)","(4, 6, 7, 8)","(1, 2, 3, 4, 5)",https://swapi.dev/api/films/1/
1,2,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,"(1, 2, 3, 4, 5, 10, 13, 14, 18, 20, 21, 22, 23...","(4, 5, 6, 27)","(3, 10, 11, 12, 15, 17, 21, 22, 23)","(8, 14, 16, 18, 19, 20)","(1, 2, 3, 6, 7)",https://swapi.dev/api/films/2/
2,3,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,"(1, 2, 3, 4, 5, 10, 13, 14, 16, 18, 20, 21, 22...","(1, 5, 7, 8, 9)","(2, 3, 10, 11, 12, 15, 17, 22, 23, 27, 28, 29)","(8, 16, 18, 19, 24, 25, 26, 30)","(1, 2, 3, 5, 6, 8, 9, 10, 15)",https://swapi.dev/api/films/3/
3,4,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,"(2, 3, 10, 11, 16, 20, 21, 32, 33, 34, 35, 36,...","(1, 8, 9)","(31, 32, 39, 40, 41)","(33, 34, 35, 36, 37, 38, 42)","(1, 2, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19, ...",https://swapi.dev/api/films/4/
4,5,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,"(2, 3, 6, 7, 10, 11, 20, 21, 22, 33, 35, 36, 4...","(1, 8, 9, 10, 11)","(21, 32, 39, 43, 47, 48, 49, 52, 58)","(4, 44, 45, 46, 50, 51, 53, 54, 55, 56, 57)","(1, 2, 6, 12, 13, 15, 28, 29, 30, 31, 32, 33, ...",https://swapi.dev/api/films/5/
5,6,Revenge of the Sith,3,War! The Republic is crumbling\r\nunder attack...,George Lucas,Rick McCallum,2005-05-19,"(1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 20, 21, ...","(1, 2, 5, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19)","(2, 32, 48, 59, 61, 63, 64, 65, 66, 68, 74, 75)","(33, 50, 53, 56, 60, 62, 67, 69, 70, 71, 72, 7...","(1, 2, 3, 6, 15, 19, 20, 23, 24, 25, 26, 27, 2...",https://swapi.dev/api/films/6/


# Normalization
Next step is to normalize the datasets in order to create the database.

In [51]:
dataframes_normalized = copy.deepcopy(dataframes)

In [52]:
columns_to_drop = {
    'films' : ['character_id', 'planet_id', 'species_id', 'vehicle_id', 'starship_id', 'url'],
    'people' : ['film_id', 'species_id', 'vehicle_id', 'starship_id', 'url'],
    'planets' : ['residents_id', 'film_id', 'url'],
    'species' : ['character_id', 'film_id', 'url'],
    'starships' : ['pilot_id', 'film_id', 'url'],
    'vehicles' : ['pilot_id', 'film_id', 'url'],
}  

### Drop the corresponding columns in order to normalize the tables

In [53]:
for cat in dataframes_normalized.keys():
    dataframes_normalized[cat].drop(columns_to_drop[cat], axis='columns', inplace = True)

## Store the normalized dataframes

In [54]:
data_path = './data'
for cat in categories:
    filename = f'{cat}_dataframe.csv'
    if os.path.exists(f'{data_path}/csv_normalized/{filename}'):
        print(f'File {filename} already exist!')
        pass
    else:
        os.makedirs(f'{data_path}/csv_normalized/', exist_ok=True)
        df = dataframes_normalized[cat]
        df.to_csv(f'{data_path}/csv_normalized/{cat}_dataframe_normalized.csv', index = False)
print(f'Dataframes of each normalized category are stored in {data_path}/csv_normalized/ as csv files!')

Dataframes of each normalized category are stored in ./data/csv_normalized/ as csv files!


## Example of joined people and their vehicles

In [ ]:
# df2 = pd.merge(people_vehicles_junction, categories_dataframes['people'], on='people_id', how = 'inner')
# df2.rename(columns={'vehicles_id_x' : 'vehicles_id'}, inplace=True)

# df2 = pd.merge(df2, categories_dataframes['vehicles'], on='vehicles_id')
# #df2.drop(['people_id', 'vehicles_id'], axis = 1)
# df2.head()

In [ ]:
# categories_dataframes['people']

# Insert data into the database

## Load database parameters from `.env` file

In [55]:
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

## Create the db connection

In [56]:
connection_string = (
    f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
)

# --- 4. Create the SQLAlchemy Engine ---
try:
    engine = create_engine(connection_string)
    print("SQLAlchemy Engine created successfully. 🛠️")
except Exception as e:
    print(f"Error creating engine: {e}")

SQLAlchemy Engine created successfully. 🛠️


## Populate the data into the database

The order of tables to be filled must be:
1. planets
2. species
3. starships
4. vehicles
5. films
6. people

In [57]:
categories_sorted = ['planets', 'species', 'vehicles', 'starships', 'films', 'people']
insert = False

### Insert the category tables into the database

In [60]:
def insert_category(cat, dictionary):
    df = dictionary[cat]
    try:
        df.to_sql(name=cat, con=engine, if_exists='append', index=False)
        print(f"DataFrame for category '{cat}' inserted successfully into the database. ✅\n")
    except Exception as e:
        print(f"\\ Error inserting DataFrame for category '{cat}': \n{e}\n\n")

In [87]:
for cat in categories_sorted:
    query = f'select * from {cat} limit 1 ;'
    # if table is empty, fill it with the corresponding data
    if pd.read_sql(query, con = engine).shape[0] == 0:
        insert_category(cat, dataframes_normalized)
    else:
        print(f'{cat} table already exists in database!')

DataFrame for category 'planets' inserted successfully into the database. ✅

DataFrame for category 'species' inserted successfully into the database. ✅

DataFrame for category 'vehicles' inserted successfully into the database. ✅

DataFrame for category 'starships' inserted successfully into the database. ✅

DataFrame for category 'films' inserted successfully into the database. ✅

DataFrame for category 'people' inserted successfully into the database. ✅



### Insert junction tables

In [89]:
for table in junction_tables_dict.keys():
    query = f'select * from {table} limit 1 ;'
    # if table is empty, fill it with the corresponding data
    if pd.read_sql(query, con = engine).shape[0] == 0:
        insert_category(table, junction_tables_dict)
    else:
        print(f'{cat} table already exists in database!') 

\ Error inserting DataFrame for category 'people_vehicles_junction': 
(pymysql.err.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`starwars_db`.`people_vehicles_junction`, CONSTRAINT `people_vehicles_junction_ibfk_2` FOREIGN KEY (`vehicle_id`) REFERENCES `vehicles` (`vehicle_id`))')
[SQL: INSERT INTO people_vehicles_junction (character_id, vehicle_id) VALUES (%(character_id)s, %(vehicle_id)s)]
[parameters: [{'character_id': 1, 'vehicle_id': 14}, {'character_id': 1, 'vehicle_id': 30}, {'character_id': 5, 'vehicle_id': 30}, {'character_id': 10, 'vehicle_id': 38}, {'character_id': 11, 'vehicle_id': 44}, {'character_id': 11, 'vehicle_id': 46}, {'character_id': 13, 'vehicle_id': 19}, {'character_id': 17, 'vehicle_id': 14}  ... displaying 10 of 13 total bound parameter sets ...  {'character_id': 69, 'vehicle_id': 45}, {'character_id': 78, 'vehicle_id': 60}]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


\ Error inserting DataFrame f